# Library

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb


from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score

# Data

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

sample_submission = pd.read_csv('../data/sample_submission.csv')

# model_Data

- Data

In [3]:
X = train.drop(['id', 'NObeyesdad'], axis=1)
y = train['NObeyesdad']

- OneHotEncoder

In [4]:
X_encoded = pd.get_dummies(X, columns=['MTRANS',
                                       'SCC',
                                       'SMOKE',
                                       'CAEC',
                                       'FAVC',
                                       'family_history_with_overweight',
                                       'Gender'])

X_encoded.head()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,CALC,MTRANS_Automobile,...,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,FAVC_no,FAVC_yes,family_history_with_overweight_no,family_history_with_overweight_yes,Gender_Female,Gender_Male
0,24.443011,1.699998,81.669950,2.000000,2.983297,2.763573,0.000000,0.976473,Sometimes,False,...,False,False,True,False,False,True,False,True,False,True
1,18.000000,1.560000,57.000000,2.000000,3.000000,2.000000,1.000000,1.000000,no,True,...,False,True,False,False,False,True,False,True,True,False
2,18.000000,1.711460,50.165754,1.880534,1.411685,1.910378,0.866045,1.673584,no,False,...,False,False,True,False,False,True,False,True,True,False
3,20.952737,1.710730,131.274851,3.000000,3.000000,1.674061,1.467863,0.780199,Sometimes,False,...,False,False,True,False,False,True,False,True,True,False
4,31.641081,1.914186,93.798055,2.679664,1.971472,1.979848,1.967973,0.931721,Sometimes,False,...,False,False,True,False,False,True,False,True,False,True


In [5]:
test_data = test.drop(['id'], axis=1)

test_encoded = pd.get_dummies(test_data, columns=['MTRANS',
                                       'SCC',
                                       'SMOKE',
                                       'CAEC',
                                       'FAVC',
                                       'family_history_with_overweight',
                                       'Gender'])

test_encoded.head()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,CALC,MTRANS_Automobile,...,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,FAVC_no,FAVC_yes,family_history_with_overweight_no,family_history_with_overweight_yes,Gender_Female,Gender_Male
0,26.899886,1.848294,120.644178,2.938616,3.000000,2.825629,0.855400,0.000000,Sometimes,False,...,False,False,True,False,False,True,False,True,False,True
1,21.000000,1.600000,66.000000,2.000000,1.000000,3.000000,1.000000,0.000000,Sometimes,False,...,False,False,True,False,False,True,False,True,True,False
2,26.000000,1.643355,111.600553,3.000000,3.000000,2.621877,0.000000,0.250502,Sometimes,False,...,False,False,True,False,False,True,False,True,True,False
3,20.979254,1.553127,103.669116,2.000000,2.977909,2.786417,0.094851,0.000000,Sometimes,False,...,False,False,True,False,False,True,False,True,False,True
4,26.000000,1.627396,104.835346,3.000000,3.000000,2.653531,0.000000,0.741069,Sometimes,False,...,False,False,True,False,False,True,False,True,True,False


In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X_encoded.CALC = le.fit_transform(X_encoded.CALC)
test_encoded.CALC = le.fit_transform(test_encoded.CALC)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_encoded, 
                                                    y, 
                                                    random_state=42, 
                                                    stratify=y)

- model

In [8]:
base_model = lgb.LGBMClassifier()
base_model.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001533 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2063
[LightGBM] [Info] Number of data points in the train set: 15568, number of used features: 28
[LightGBM] [Info] Start training from score -2.107583
[LightGBM] [Info] Start training from score -1.907537
[LightGBM] [Info] Start training from score -1.964517
[LightGBM] [Info] Start training from score -1.854860
[LightGBM] [Info] Start training from score -1.635336
[LightGBM] [Info] Start training from score -2.146381
[LightGBM] [Info] Start training from score -2.107583


LGBMClassifier()

# val予測

In [9]:
y_val_pred = base_model.predict(X_val)

In [10]:
accuracy = accuracy_score(y_val, y_val_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9090558766859345


# test予測

In [11]:
test_pred = base_model.predict(test_encoded)

- 結合

In [12]:
sample_submission['NObeyesdad'] = test_pred
sample_submission.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III


In [13]:
sample_submission.to_csv(f'../output/submit_val{accuracy:.4f}.csv', index=False)

- submit確認

In [14]:
sample_submission.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
